In [2]:
# warnings kütüphanesi uyarıları kapatmak için kullanılır
import warnings 
warnings.filterwarnings("ignore")

In [3]:
# GPU ve CUDA sürümlerini kontrol etmek için kullanılır
!nvidia-smi

Mon Jun 24 20:19:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q -U transformers torch==2.2.1 datasets 
!echo "Installations completed!"

In [6]:
import numpy as np 
import pandas as pd 
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline 

from sklearn.metrics import (accuracy_score, 
                             precision_score, 
                             recall_score, 
                             f1_score)

## 🤗 Hugging Face Datasets Kütüphanesi

### Parametreler

- **`path`**:
  - Yüklenecek veri setinin yolu veya ismi. Örnek: "imdb", "glue".

- **`name`**:
  - Yüklenecek veri setinin alt kümesi. Örnek: "sst2" (GLUE için).

- **`data_dir`**:
  - Veri dosyalarının bulunduğu dizin.

- **`data_files`**:
  - Yüklenecek veri dosyaları.

- **`split`**:
  - Veri setinin bölünmesi (örneğin "train", "test").

- **`cache_dir`**:
  - Verinin önbelleğe alınacağı dizin.

- **`features`**:
  - Özelliklerin açıkça belirtildiği yer.

- **`download_config`**:
  - İndirme yapılandırma ayarları.

- **`download_mode`**:
  - İndirme modu: "reuse_dataset_if_exists", "reuse_cache_if_exists", "force_redownload".

- **`verification_mode`**:
  - Veri setinin doğrulama modu.

- **`ignore_verifications`**:
  - Artık kullanılmıyor, doğrulamaları atlamak için.

- **`keep_in_memory`**:
  - Eğer True ise, veri seti bellekte tutulur.

- **`save_infos`**:
  - Eğer True ise, veri seti bilgileri kaydedilir.

- **`revision`**:
  - Yüklenecek veri setinin versiyonu veya commit ID'si.

- **`token`**:
  - Private veri setleri için kullanılır, bir token sağlar.

- **`use_auth_token`**:
  - Artık kullanılmıyor, oturum açma token'ı için.

- **`task`**:
  - Artık kullanılmıyor, veri seti yükleme görevini belirtmek için.

- **`streaming`**:
  - Eğer True ise, veri seti akış modunda yüklenir.

- **`num_proc`**:
  - Çok işlemcili veri işleme için işlemci sayısı.

- **`storage_options`**:
  - Depolama seçenekleri.

- **`trust_remote_code`**:
  - Eğer True ise, uzaktan kod çalıştırmaya izin verir.

- **`**config_kwargs`**:
  - Diğer ek yapılandırma argümanları.


In [7]:
# Veri setini yükleme
dataset = load_dataset(path="stanfordnlp/imdb", split='train')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
print(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [10]:
df = dataset.to_pandas()

df.tail()

,text,label
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1
24999,The story centers around Barry McKenzie who mu...,1


In [14]:
num_labels = len(df['label'].unique())
print(f"Num Labels: {num_labels}")

df['label'].value_counts()

Num Labels: 2


label
0    12500
1    12500
Name: count, dtype: int64

In [16]:
train_data, _ = train_test_split(df, train_size=0.1, stratify=df['label'])
test_data, _ = train_test_split(df, test_size=0.02, stratify=df['label'])

train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

In [20]:
print(f"Type : {type(train_data)}")
print("===========")
print(train_data[0])

Type : <class 'datasets.arrow_dataset.Dataset'>
{'text': "In director Eric Stanze's 'ISOYC, IPOYG', three men are subjected to torture at the the hands of a woman that they have all sexually abused. The first victim is forced to eat his own crap, before being axed to death. The next bloke ends up with a bullet in the crotch after refusing to have anal sex with the first guy's corpse. But it's the third man who gets it the worst: he has to watch the heavily tattooed 'star' Emily Haack get naked and masturbate with a broom handle (oh, he also gets the handle shoved up his butt too!).<br /><br />And, unfortunately, so do we (get to see her masturbate, that is\x97not get a broom handle up our butts!).<br /><br />Yes, 'ISOYC, IPOYG' is one harsh viewing experience, not because of its relentless violence, but because Haack, who is obviously under the misguided notion that she has the body of a goddess (as opposed to that of a roadie for Metallica) constantly gets buck naked for the camera. I

In [21]:
base_model_name = 'distilbert-base-uncased'

# Tokenizer yükleme        
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## 🤗 Hugging Face Tokenizer Kütüphanesi

### Parametreler

- **`text`**:
  - Tokenize edilecek metin veya metinlerin listesi.

- **`text_pair`**:
  - İkinci bir metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`text_target`**:
  - Hedef metin veya metinlerin listesi (genellikle seq2seq modelleri için kullanılır).

- **`text_pair_target`**:
  - Hedef ikinci metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`add_special_tokens`**:
  - Özel tokenler ekler (örneğin [CLS], [SEP]). True olması, modeli daha iyi performans gösterir.

- **`padding`**:
  - Padding stratejisi: True, False, "longest", "max_length".

- **`truncation`**:
  - Kesme stratejisi: True, False, "longest_first", "only_first".

- **`max_length`**:
  - Maksimum token sayısı. Yüksek değerler daha fazla bilgi taşır ama daha fazla bellek kullanır.

- **`stride`**:
  - Kesme sırasında kayma boyutu. Uzun metinler için daha küçük değerler kullanışlı olabilir.

- **`is_split_into_words`**:
  - Eğer True ise, metin kelimelere bölünmüş olarak kabul edilir.

- **`pad_to_multiple_of`**:
  - Padding boyutunun bir katı olacak şekilde padding ekler.

- **`return_tensors`**:
  - Döndürülecek tensör tipi: 'pt', 'tf', 'np'.

- **`return_token_type_ids`**:
  - Eğer True ise, token tipi ID'lerini döndürür.

- **`return_attention_mask`**:
  - Eğer True ise, attention maskelerini döndürür.

- **`return_overflowing_tokens`**:
  - Eğer True ise, taşan tokenleri döndürür.

- **`return_special_tokens_mask`**:
  - Eğer True ise, özel token maskesini döndürür.

- **`return_offsets_mapping`**:
  - Eğer True ise, offset mapping döndürür.

- **`return_length`**:
  - Eğer True ise, token uzunluklarını döndürür.

- **`verbose`**:
  - Eğer True ise, işlem hakkında daha fazla bilgi verir.

- **`**kwargs`**:
  - Diğer ek argümanlar.


In [22]:
def text_tokenizer(text):
    """
    text_tokenizer: Bu fonksiyon verilen metni tokenize eder.

    Args:
        text: str: Tokenize edilecek metin
    """
    return tokenizer(
        text['text'], 
        padding='max_length',
        truncation=True
    )

train_data = train_data.map(
    text_tokenizer, 
    batched=True
)

test_data = test_data.map(
    text_tokenizer, 
    batched=True
)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/24500 [00:00<?, ? examples/s]

In [23]:
print(train_data)

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 2500
})


In [25]:
print(train_data[0])

{'text': "In director Eric Stanze's 'ISOYC, IPOYG', three men are subjected to torture at the the hands of a woman that they have all sexually abused. The first victim is forced to eat his own crap, before being axed to death. The next bloke ends up with a bullet in the crotch after refusing to have anal sex with the first guy's corpse. But it's the third man who gets it the worst: he has to watch the heavily tattooed 'star' Emily Haack get naked and masturbate with a broom handle (oh, he also gets the handle shoved up his butt too!).<br /><br />And, unfortunately, so do we (get to see her masturbate, that is\x97not get a broom handle up our butts!).<br /><br />Yes, 'ISOYC, IPOYG' is one harsh viewing experience, not because of its relentless violence, but because Haack, who is obviously under the misguided notion that she has the body of a goddess (as opposed to that of a roadie for Metallica) constantly gets buck naked for the camera. It ain't a pretty sight.<br /><br />In addition t

## 🤗 Hugging Face TrainingArguments Kütüphanesi

### Parametreler

- **`output_dir`**:
  - Eğitim sırasında ve sonrasında sonuçların kaydedileceği dizin.

- **`overwrite_output_dir`**:
  - Eğer True ise, mevcut `output_dir` içeriğini üzerine yazar.

- **`do_train`**:
  - Eğitim işleminin yapılıp yapılmayacağını belirtir.

- **`do_eval`**:
  - Değerlendirme işleminin yapılıp yapılmayacağını belirtir.

- **`do_predict`**:
  - Tahmin işleminin yapılıp yapılmayacağını belirtir.

- **`eval_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`prediction_loss_only`**:
  - Eğer True ise, sadece kayıp hesaplanır, tahminler döndürülmez.

- **`per_device_train_batch_size`**:
  - Her bir cihaz (GPU/CPU) için eğitim batch boyutu.

- **`per_device_eval_batch_size`**:
  - Her bir cihaz (GPU/CPU) için değerlendirme batch boyutu.

- **`gradient_accumulation_steps`**:
  - Gradient biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama eğitim süresini uzatır.

- **`eval_accumulation_steps`**:
  - Değerlendirme biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama değerlendirme süresini uzatır.

- **`eval_delay`**:
  - Eğitim başladıktan sonra ilk değerlendirmenin yapılacağı adım sayısı.

- **`learning_rate`**:
  - Öğrenme oranı. Yüksek değerler daha hızlı öğrenir ama aşırı öğrenmeye (overfitting) yol açabilir.

- **`weight_decay`**:
  - Ağırlıkların küçülme oranı. Yüksek değerler modelin genelleme yeteneğini artırabilir ama öğrenme yavaşlar.

- **`adam_beta1`**:
  - Adam optimizasyon algoritması için beta1 parametresi.

- **`adam_beta2`**:
  - Adam optimizasyon algoritması için beta2 parametresi.

- **`adam_epsilon`**:
  - Adam optimizasyon algoritması için epsilon parametresi.

- **`max_grad_norm`**:
  - Gradientlerin maksimum normu. Büyük değerler modelin stabilitesini artırabilir.

- **`num_train_epochs`**:
  - Eğitim için epoch sayısı. Yüksek değerler daha fazla öğrenme sağlar ama aşırı öğrenmeye yol açabilir.

- **`max_steps`**:
  - Maksimum eğitim adım sayısı. -1 ise, tüm epoch'lar tamamlanır.

- **`lr_scheduler_type`**:
  - Öğrenme oranı zamanlayıcı tipi.

- **`lr_scheduler_kwargs`**:
  - Öğrenme oranı zamanlayıcısı için ek parametreler.

- **`warmup_ratio`**:
  - Öğrenme oranı ısınma oranı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`warmup_steps`**:
  - Öğrenme oranı ısınma adım sayısı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`log_level`**:
  - Log seviyesi. 'passive', 'info', 'warning', 'error' veya 'critical'.

- **`log_level_replica`**:
  - Çoklu GPU eğitiminde log seviyesi.

- **`log_on_each_node`**:
  - Çoklu node eğitiminde her node için loglama yapılır.

- **`logging_dir`**:
  - TensorBoard logları için dizin.

- **`logging_strategy`**:
  - Loglama stratejisi: 'no', 'steps' veya 'epoch'.

- **`logging_first_step`**:
  - Eğer True ise, ilk adımda loglama yapılır.

- **`logging_steps`**:
  - Kaç adımda bir loglama yapılacağı.

- **`logging_nan_inf_filter`**:
  - Eğer True ise, NaN ve sonsuz değerler loglanmaz.

- **`save_strategy`**:
  - Modelin kaydedilme stratejisi: 'no', 'steps' veya 'epoch'.

- **`save_steps`**:
  - Kaç adımda bir modelin kaydedileceği.

- **`save_total_limit`**:
  - Maksimum kaç model kaydedileceği.

- **`save_safetensors`**:
  - Modelin güvenli tensör formatında kaydedilip kaydedilmeyeceği.

- **`save_on_each_node`**:
  - Eğer True ise, her node'da model kaydedilir.

- **`save_only_model`**:
  - Eğer True ise, sadece model kaydedilir, optimizer ve lr scheduler kaydedilmez.

- **`restore_callback_states_from_checkpoint`**:
  - Callback durumlarının checkpoint'ten geri yüklenip yüklenmeyeceği.

- **`no_cuda`**:
  - Eğer True ise, GPU kullanılmaz.

- **`use_cpu`**:
  - Eğer True ise, CPU kullanılır.

- **`use_mps_device`**:
  - Eğer True ise, MacOS Metal Performance Shaders kullanılır.

- **`seed`**:
  - Rastgelelik için seed değeri.

- **`data_seed`**:
  - Veri yükleme için seed değeri.

- **`jit_mode_eval`**:
  - Eğer True ise, PyTorch JIT modu kullanılır.

- **`use_ipex`**:
  - Eğer True ise, Intel Extension for PyTorch kullanılır.

- **`bf16`**:
  - Eğer True ise, bfloat16 kullanılır.

- **`fp16`**:
  - Eğer True ise, float16 kullanılır.

- **`fp16_opt_level`**:
  - float16 optimizasyon seviyesi.

- **`half_precision_backend`**:
  - Yarı hassasiyet arka ucu: 'auto', 'amp' veya 'apex'.

- **`bf16_full_eval`**:
  - Eğer True ise, bfloat16 tam değerlendirme yapılır.

- **`fp16_full_eval`**:
  - Eğer True ise, float16 tam değerlendirme yapılır.

- **`tf32`**:
  - Eğer True ise, TensorFloat-32 kullanılır.

- **`local_rank`**:
  - Çoklu GPU eğitiminde lokal rank.

- **`ddp_backend`**:
  - DDP backend tipi: 'nccl', 'gloo', 'mpi'.

- **`tpu_num_cores`**:
  - TPU çekirdek sayısı.

- **`tpu_metrics_debug`**:
  - TPU metriklerinin debug bilgilerini içerir.

- **`debug`**:
  - Debug seçeneği: 'underflow_overflow', 'poor_optimization'.

- **`dataloader_drop_last`**:
  - Eğer True ise, dataloader son batch'i düşürür.

- **`eval_steps`**:
  - Değerlendirme adım sayısı.

- **`dataloader_num_workers`**:
  - Dataloader için çalışan sayısı. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`dataloader_prefetch_factor`**:
  - Dataloader için prefetch faktörü. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`past_index`**:
  - Geçmiş index.

- **`run_name`**:
  - Çalışma ismi.

- **`disable_tqdm`**:
  - Eğer True ise, tqdm progress bar devre dışı bırakılır.

- **`remove_unused_columns`**:
  - Eğer True ise, kullanılmayan sütunlar veri setinden kaldırılır.

- **`label_names`**:
  - Label isimleri.

- **`load_best_model_at_end`**:
  - Eğitim sonunda en iyi modelin yüklenip yüklenmeyeceği.

- **`metric_for_best_model`**:
  - En iyi modelin seçilmesi için kullanılacak metrik.

- **`greater_is_better`**:
  - Eğer True ise, metriklerde yüksek değerler daha iyi olarak kabul edilir.

- **`ignore_data_skip`**:
  - Eğer True ise, veri atlamaları göz ardı edilir.

- **`fsdp`**:
  - Fully Sharded Data Parallel ayarları.

- **`fsdp_min_num_params`**:
  - Fully Sharded Data Parallel için minimum parametre sayısı.

- **`fsdp_config`**:
  - Fully Sharded Data Parallel için ek ayarlar.

- **`fsdp_transformer_layer_cls_to_wrap`**:
  - Fully Sharded Data Parallel için transformer layer sınıfı.

- **`accelerator_config`**:
  - Accelerator için ek ayarlar.

- **`deepspeed`**:
  - DeepSpeed yapılandırması.

- **`label_smoothing_factor`**:
  - Label smoothing faktörü.

- **`optim`**:
  - Optimizasyon algoritması.

- **`optim_args`**:
  - Optimizasyon algoritması için ek argümanlar.

- **`adafactor`**:
  - Eğer True ise, Adafactor optimizasyon algoritması kullanılır.

- **`group_by_length`**:
  - Eğer True ise, input uzunluğuna göre gruplanır.

- **`length_column_name`**:
  - Uzunluk sütunu ismi.

- **`report_to`**:
  - Raporlama platformları (örneğin 'wandb').

- **`ddp_find_unused_parameters`**:
  - DDP'de kullanılmayan parametrelerin bulunması.

- **`ddp_bucket_cap_mb`**:
  - DDP bucket kapasitesi.

- **`ddp_broadcast_buffers`**:
  - DDP broadcast buffer'ları.

- **`dataloader_pin_memory`**:
  - Dataloader için bellek pinleme.

- **`dataloader_persistent_workers`**:
  - Dataloader için kalıcı çalışanlar.

- **`skip_memory_metrics`**:
  - Eğer True ise, bellek metrikleri atlanır.

- **`use_legacy_prediction_loop`**:
  - Eğer True ise, eski tahmin döngüsü kullanılır.

- **`push_to_hub`**:
  - Eğer True ise, model Hugging Face Hub'a itilir.

- **`resume_from_checkpoint`**:
  - Checkpoint'ten devam edilir.

- **`hub_model_id`**:
  - Hugging Face Hub model ID.

- **`hub_strategy`**:
  - Hub stratejisi.

- **`hub_token`**:
  - Hugging Face Hub token.

- **`hub_private_repo`**:
  - Eğer True ise, özel repo kullanılır.

- **`hub_always_push`**:
  - Eğer True ise, her zaman Hub'a itilir.

- **`gradient_checkpointing`**:
  - Eğer True ise, gradient checkpointing yapılır.

- **`gradient_checkpointing_kwargs`**:
  - Gradient checkpointing için ek argümanlar.

- **`include_inputs_for_metrics`**:
  - Eğer True ise, değerlendirme metrikleri için inputlar da dahil edilir.

- **`eval_do_concat_batches`**:
  - Eğer True ise, değerlendirme batch'leri birleştirilir.

- **`fp16_backend`**:
  - float16 arka ucu.

- **`evaluation_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`push_to_hub_model_id`**:
  - Hub model ID.

- **`push_to_hub_organization`**:
  - Hub organizasyon ID.

- **`push_to_hub_token`**:
  - Hub token.

- **`mp_parameters`**:
  - Model paralel parametreler.

- **`auto_find_batch_size`**:
  - Eğer True ise, batch boyutu otomatik bulunur.

- **`full_determinism`**:
  - Eğer True ise, deterministik eğitim yapılır.

- **`torchdynamo`**:
  - TorchDynamo kullanımı.

- **`ray_scope`**:
  - Ray scope.

- **`ddp_timeout`**:
  - DDP timeout süresi.

- **`torch_compile`**:
  - Eğer True ise, Torch compile kullanılır.

- **`torch_compile_backend`**:
  - Torch compile backend.

- **`torch_compile_mode`**:
  - Torch compile modu.

- **`dispatch_batches`**:
  - Batch dispatching.

- **`split_batches`**:
  - Batch splitting.

- **`include_tokens_per_second`**:
  - Eğer True ise, saniye başına token sayısı dahil edilir.

- **`include_num_input_tokens_seen`**:
  - Eğer True ise, görülen input token sayısı dahil edilir.

- **`neftune_noise_alpha`**:
  - Neptune noise alpha.

- **`optim_target_modules`**:
  - Optimizasyon hedef modüller.

- **`batch_eval_metrics`**:
  - Eğer True ise, batch değerlendirme metrikleri hesaplanır.


In [11]:
# Modeli yükleme        
model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# TrainingArguments sınıfı ile modelin eğitim parametrelerini belirleme
training_args = TrainingArguments(
    output_dir="./results", 
    evaluation_strategy="steps", 
    save_strategy="steps",  
    learning_rate=2e-5, 
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,  
    num_train_epochs=1, 
    weight_decay=0.01, 
    logging_dir='./logs',
    logging_steps=10,  
    save_steps=10,
    save_total_limit=2,  
    load_best_model_at_end=True,  
    metric_for_best_model="accuracy", 
    greater_is_better=True,  
    do_train=True,  
    do_eval=True,  
    report_to="none"  
)

In [13]:
def compute_metrics(p):
    """
    compute_metrics: Bu fonksiyon modelin performansını değerlendirmek için kullanılır.

    Args:
        p: Trainer: Trainer sınıfı, modelin tahminlerini ve etiketlerini içerir.
    """
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

## 🤗 Hugging Face Trainer Kütüphanesi

### Parametreler

- **`model`**:
  - Eğitim ve değerlendirme için kullanılacak model.

- **`args`**:
  - Eğitim argümanları, `TrainingArguments` sınıfı ile oluşturulur.

- **`data_collator`**:
  - Veriyi toplamak için kullanılan data collator.

- **`train_dataset`**:
  - Eğitim için kullanılacak veri seti.

- **`eval_dataset`**:
  - Değerlendirme için kullanılacak veri seti.

- **`tokenizer`**:
  - Metin verilerini modelin anlayabileceği sayısal değerlere dönüştürür.

- **`model_init`**:
  - Modelin başlatılması için kullanılacak fonksiyon.

- **`compute_metrics`**:
  - Değerlendirme metriklerini hesaplamak için kullanılacak fonksiyon.

- **`callbacks`**:
  - Eğitim sürecini izlemek ve kontrol etmek için kullanılacak callback listesi.

- **`optimizers`**:
  - Optimizasyon algoritması ve öğrenme oranı zamanlayıcısı.

- **`preprocess_logits_for_metrics`**:
  - Metrik hesaplama için logits'i ön işleme fonksiyonu.


In [14]:
# Trainer sınıfı ile modelin eğitim için hazırlanması
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data, 
    compute_metrics=compute_metrics
)

In [15]:
# Modelin eğitilmesi
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
10,0.695000,0.685610,0.532776,0.652185,0.532776,0.418760
20,0.695800,0.671482,0.578776,0.742546,0.578776,0.493231
30,0.657300,0.632409,0.802367,0.813940,0.802367,0.800529
40,0.598200,0.541805,0.817469,0.828717,0.817469,0.815894
50,0.515600,0.439165,0.851673,0.853712,0.851673,0.851459
60,0.498000,0.374981,0.871184,0.871726,0.871184,0.871137
70,0.406900,0.345650,0.874245,0.876447,0.874245,0.874061
80,0.327100,0.313198,0.883429,0.883677,0.883429,0.883410
90,0.307900,0.301396,0.885918,0.886586,0.885918,0.885869
100,0.347100,0.340858,0.858694,0.871967,0.858694,0.857422


TrainOutput(global_step=157, training_loss=0.443889757630172, metrics={'train_runtime': 3882.3274, 'train_samples_per_second': 0.644, 'train_steps_per_second': 0.04, 'total_flos': 331168496640000.0, 'train_loss': 0.443889757630172, 'epoch': 1.0})

In [95]:
texts = [
    "This movie was fantastic! The performances were oscar-worthy.",
    "I did not like the film. The plot was too predictable.",
    "An average movie, some good parts, but overall not very impressive.",
    "Absolutely loved this film from start to finish. The cinematography was stunning and the storytelling was captivating.",
    "Worst movie I have seen in years! It was a complete waste of time and money.",
    "It was okay, not great. Some scenes were entertaining, but the script felt weak.",
    "A masterpiece of modern cinema, beautifully crafted and deeply moving.",
    "I found the movie to be mediocre at best. It lacked originality and depth.",
    "Hilarious from start to finish! A must-watch for anyone who loves a good comedy.",
    "Disappointing experience, the film did not live up to the hype at all.",
    "An inspiring and thought-provoking film that challenges societal norms.",
    "The film had a slow start, but it really picked up in the second half and ended strong.",
    "A visually stunning film, but the storyline was too complex and hard to follow."
]


In [96]:
text_data = [{'text' : text} for text in texts] # Tokenize edilecek metinlerin listesi
text_data[0]

{'text': 'This movie was fantastic! The performances were oscar-worthy.'}

In [97]:
tokenized_texts = [text_tokenizer(text) for text in text_data] # Metinlerin tokenize edilmesi
tokenized_texts[0]

{'input_ids': [101, 2023, 3185, 2001, 10392, 999, 1996, 4616, 2020, 7436, 1011, 11007, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [98]:
tokenized_data = {key : [] for key in tokenized_texts[0].keys()} # Tokenize edilmiş metinlerin sözlük yapısında saklanması
tokenized_data

{'input_ids': [], 'attention_mask': []}

In [99]:
# Tokenize edilmiş metinlerin sözlüğe eklenmesi
for item in tokenized_texts: 
    for key, value in item.items():
        tokenized_data[key].append(value)
        
# Tokenize edilmiş metinlerin Dataset sınıfına dönüştürülmesi
predict_dataset = Dataset.from_dict(tokenized_data)
predict_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 13
})

In [100]:
predictions = trainer.predict(predict_dataset) # Modelin tahmin yapması
preds = np.argmax(predictions.predictions, axis=1) # Tahminlerin sınıflara dönüştürülmesi
preds

array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0])

In [101]:
labels = ['Positive' if pred == 1 else 'Negative' for pred in preds]

for text, label in zip(texts, labels):
    print(f"Text: {text}\nPrediction: {label}\n")

Text: This movie was fantastic! The performances were oscar-worthy.

Prediction: Positive



Text: I did not like the film. The plot was too predictable.

Prediction: Negative



Text: An average movie, some good parts, but overall not very impressive.

Prediction: Positive



Text: Absolutely loved this film from start to finish. The cinematography was stunning and the storytelling was captivating.

Prediction: Positive



Text: Worst movie I have seen in years! It was a complete waste of time and money.

Prediction: Negative



Text: It was okay, not great. Some scenes were entertaining, but the script felt weak.

Prediction: Negative



Text: A masterpiece of modern cinema, beautifully crafted and deeply moving.

Prediction: Positive



Text: I found the movie to be mediocre at best. It lacked originality and depth.

Prediction: Negative



Text: Hilarious from start to finish! A must-watch for anyone who loves a good comedy.

Prediction: Positive



Text: Disappointing experience, 